# Raster masks from vector masks

[xsar.Sentinel1Dataset.dataset](../basic_api.rst#xsar.Sentinel1Dataset.dataset) has a `land_mask` variable by default, rasterized from [cartopy.feature.NaturalEarthFeature('physical', 'land', '10m')](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#cartopy.feature.NaturalEarthFeature)



## Mask sigma0 over land

Replacing `sigma0` values with `nan` over ocean can be done with `xarray.where`

In [ ]:
import xarray as xr
import xsar
import numpy as np
import os
ds_xsar = xsar.open_dataset(xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE'), resolution='1000m')
ds = ds_xsar['measurement'].ds
# apply land_mask. The final transpose is done to preserve dimensions ordering
ds['sigma0_land'] = xr.where(ds['land_mask'], ds['sigma0'], np.nan).transpose(*ds['sigma0'].dims)
ds['sigma0_land']


## Adding masks

Masks can be added with [xsar.Sentinel1Meta.set_mask_feature](../basic_api.rst#xsar.Sentinel1Meta.set_mask_feature), providing a shapefile or a [cartopy.feature.Feature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) object.

For a default mask for all SAFE, use classmethod `xsar.Sentinel1Meta.set_mask_feature` 

Here, we add a `ocean_mask` dataset variable:

In [ ]:
xsar.Sentinel1Meta.set_mask_feature('ocean', os.path.join(xsar.get_test_file('water-polygons-split-4326'), 'water_polygons.shp'))


s1meta = xsar.Sentinel1Meta(xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE'))
ds_xsar = xsar.open_dataset(s1meta, resolution='200m')
ds = ds_xsar['measurement'].ds
ds['sigma0_ocean'] = xr.where(ds['ocean_mask'], ds['sigma0'], np.nan).transpose(*ds['sigma0'].dims)
ds[['sigma0_ocean','ocean_mask']]


Masks are available as a shapely object (lon/lat coordinates), with [xsar.Sentinel1Meta.get_mask](../basic_api.rst#xsar.Sentinel1Meta.get_mask):

When using this `s1meta` object with  [xsar.Sentinel1Meta.footprint](../basic_api.rst#xsar.Sentinel1Meta.footprint), mask are rasterized over the dataset, and variables postfixed with `_mask` are created.

## Convert a mask to dataset coordinates

[xsar.Sentinel1Meta.ll2coords](../basic_api.rst#xsar.Sentinel1Meta.ll2coords) allow converting lon/lat coordinates on shapely objects.

So it's possible to plot the vector mask on a raster variable:

In [ ]:
import holoviews as hv
import geoviews.feature as gf
hv.extension('bokeh')
from holoviews.operation.datashader import datashade,rasterize


# get the shapely polygon of the mask (on the footprint, lon/lat)
land_poly = s1meta.get_mask('ocean')

# convert lon/lat to line/sample
land_poly_coords = s1meta.ll2coords(s1meta.get_mask('ocean'))

# plot masked sigma0, with coastline

(
    hv.Image(ds['sigma0_ocean'].sel(pol='VV')).opts(cmap='gray',clim=(0,0.2), colorbar=True,tools=['hover'],title="xsar") \
    * hv.Path(land_poly_coords).opts(color='lightgreen',width=800,height=800) 
).opts(invert_axes=True)